# <font color='blue'>Project - NLP: Sentiment Analysis : Twitter US Airline Sentiment</font>


*Prasad Menon*

**Data Description:** 


A sentiment analysis job about the problems of each major U.S. airline. Twitter data was scraped from February of 2015 and contributors were asked to first classify positive, negative, and neutral tweets, followed by categorizing negative reasons (such as "late flight" or "rude service"). 

**Dataset** 


The project is from a dataset from Kaggle.  
Link to the Kaggle project site: https://www.kaggle.com/crowdflower/twitter-airline-sentiment  
The dataset has to be downloaded from the above Kaggle website. 


The dataset has the following columns: 

*	tweet_id
*	airline_sentiment
*	airline_sentiment_confidence
*	negativereason 
*	negativereason_confidence 
*	airline 
*	airline_sentiment_gold 
*	name 
*	negativereason_gold 
*	retweet_count 
*	text 
*	tweet_coord 
*	tweet_created 
*	tweet_location 
*	user_timezone

**Objective:** 


To implement the techniques learnt as a part of the course. 

**Learning Outcomes:**


*	Basic understanding of text pre-processing. 
*	What to do after text pre-processing:  
*		Bag of words 
*		Tf-idf 
*	Build the classification model. 
*	Evaluate the Model.

# <font color='blue'>Step 1</font>

**Import the libraries, load dataset, print shape of data, data description**

**Import the libraries**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import re, string, unicodedata
from bs4 import BeautifulSoup

!pip install contractions
import nltk
import contractions

nltk.download('wordnet')
nltk.download('punkt')

from nltk.tokenize import word_tokenize 
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from mlxtend.plotting import plot_confusion_matrix

from imblearn.over_sampling import SMOTE


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**Load dataset**

In [ ]:
#Tweet= pandas.read_csv("../input/Tweets.csv")
#Tweet.head()

project_path =  '/kaggle/input/twitter-airline-sentiment/'

# Load the dataset
tweet_data = pd.read_csv(project_path + 'Tweets.csv',header=0)

In [ ]:
tweet_data.head()

**Shape of data**

In [ ]:
# There are 14640 rows and 15 columns in the tweet data
print(tweet_data.shape)

In [ ]:
tweet_data.info()

In [ ]:
tweet_data.dtypes

**Checking if data is duplicated and removing duplicates**

In [ ]:
# Check for duplicate rows
duplicateRowsDF = tweet_data[tweet_data.duplicated()]
print(duplicateRowsDF.head())
print(duplicateRowsDF.shape)  
# There are 36 duplicate rows

In [ ]:
# Drop the duplicate rows
tweet_data.drop_duplicates(keep=False,inplace=True) 

In [ ]:
# There are 14568 rows and 15 columns after dropping 36 duplicate rows from the initial load dataset
print(tweet_data.shape)

**Description of Data**

In [ ]:
tweet_data.describe()

**Explorotary data Analysis   EDA**

**Sentiment Analysis for each Airline**

In [ ]:
g = sns.FacetGrid(tweet_data, col="airline", col_wrap=3, height=5, aspect =0.7)
g = g.map(sns.countplot, "airline_sentiment",order =tweet_data.airline_sentiment.value_counts().index, palette='plasma')
plt.show()
# Here we can see that United Airlines, US Airways, American Airlines has the most number of negative review
# Virgin America has the least number of negative reviews

**Most Common negative review reasons**

In [ ]:
# Check the most common negative reason 
y = tweet_data['negativereason']
print(y.value_counts())
plt.figure(figsize=(25,5)) 
g = sns.countplot(y)
# Customer service and Late flight seems to be the main reason why customers are giving bad feedback

**Summary Part 1**

---



*   There are 14640 rows and 15 columns in the tweet data
*   There are 36 duplicate records in the data set
*   Here we can see that United Airlines, US Airways, American Airlines has the most number of negative review
*   Virgin America has the least number of negative reviews
*   Customer service and Late flight seems to be the main reason why customers are giving bad feedback



# <font color='blue'>Step 2</font>

**Understand of data-columns:**

a. Drop all other columns except “text” and “airline_sentiment”. 

b. Check the shape of data. 

c. Print first 5 rows of data. 

In [ ]:
tweet_data.columns

**Drop Irrelevant columns**

In [ ]:
# Let us now remove irrelevant columns
tweet_data_relevant = tweet_data.drop(['tweet_id', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count','tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'], axis =1)

**Shape of Data**

In [ ]:
# There are 14568 rows and 2 columns (This is result of keeping relevant rows and duplicate data cleanup)
tweet_data_relevant.shape

**Printing first 5 rows of data**

In [ ]:
tweet_data_relevant.head(5)

In [ ]:
print(tweet_data_relevant.airline_sentiment.value_counts())

**Is the data balanced**

In [ ]:
y = tweet_data_relevant['airline_sentiment']
print(y.value_counts())
plt.figure(figsize=(20,5)) 
g = sns.countplot(y)
# No, Here we can see that the data is not balanced, There are lot of negative sentiments

**Summary Part 2**

*   There data is not balanced, The percentage of negative sentiment is high
*   There are 14568 rows in the dataset after duplicate data cleanup

# <font color='blue'>Step 3</font>

**Text pre-processing: Data preparation. **

*	a. Html tag removal. 
*	b. Tokenization. 
*	c. Remove the numbers. 
*	d. Removal of Special Characters and Punctuations. 
*	e. Conversion to lowercase. 
*	f. Lemmatize or stemming. 
*	g. Join the words in the list to convert back to text string in the dataframe. (So that each row contains the data in text format.) 
*	h. Print first 5 rows of data after pre-processing.

**Html tag removal**

In [ ]:
def perform_html_cleanup( raw_review ):
  # 1. Remove HTML
  review_text = BeautifulSoup(raw_review).get_text()
  return review_text

**Replace Contraction**

In [ ]:
def replace_contractions(raw_review):
    #Replace contractions in raw_review
    return contractions.fix(raw_review)

**Tokenization**

In [ ]:
def perform_tokenization( raw_review ):
  # 2. Perform Tokenization
  word_tokens = word_tokenize(raw_review)  # Tokenization
  return word_tokens

**Removal of Numbers**

In [ ]:
def remove_numbers(list_of_words): 
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list_of_words] 
    return list

**Remove special characters and punctuations**

In [ ]:
def remove_special_character_punctuation(list_of_words): 
    pattern = '[^A-Za-z0-9]+'
    list = [re.sub(pattern, '', i) for i in list_of_words] 
    return list

In [ ]:
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []                        # Create empty list to store pre-processed words.
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)    # Append processed words to new list.
    return new_words

**Conversion to Lower case**

In [ ]:
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []                        # Create empty list to store pre-processed words.
    for word in words:
        new_word = word.lower()           # Converting to lowercase
        new_words.append(new_word)        # Append processed words to new list.
    return new_words

**Remove empty String**

In [ ]:
def remove_empty_string(words):
  return list(filter(None, words))

**Stemming**

In [ ]:
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []                            # Create empty list to store pre-processed words.
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)                # Append processed words to new list.
    return stems

**Lemmatization**

In [ ]:
def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []                           # Create empty list to store pre-processed words.
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)              # Append processed words to new list.
    return lemmas

**Complete Pre-preocessing**

In [ ]:
def perform_cleanup(raw_review):
    clean_review = perform_html_cleanup(raw_review)
    clean_review = replace_contractions(clean_review)
    clean_review = perform_tokenization(clean_review)
    clean_review = remove_numbers(clean_review)
    clean_review = remove_special_character_punctuation(clean_review)
    clean_review = remove_punctuation(clean_review)
    clean_review  = to_lowercase(clean_review)
    clean_review = remove_empty_string(clean_review)
    #clean_review = stem_words(clean_review)
    clean_review = lemmatize_verbs(clean_review)
    return clean_review

In [ ]:
print(tweet_data_relevant.head())

**Join the words in the list to convert back to text string in the dataframe**


(So that each row contains the data in text format.)

In [ ]:
clean_reviews = []

for i, row in tweet_data_relevant.iterrows():
    words = tweet_data_relevant.at[i, 'text']
    words = perform_cleanup(words)
    tweet_data_relevant.at[i,'text'] = " ".join( words )
    clean_reviews.append( tweet_data_relevant.at[i, 'text'] )
tweet_data_relevant.head()

**Print first 5 rows of data after pre-processing**

In [ ]:
tweet_data_relevant.head(5)

# <font color='blue'>Step 4</font>

**Vectorization: )**


a. Use CountVectorizer.

b. Use TfidfVectorizer

**CountVectorizer**

In [ ]:
print ("Creating the bag of words...\n")
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
count_vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
count_vectorizer_data_features = count_vectorizer.fit_transform(clean_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
count_vectorizer_data_features = count_vectorizer_data_features.toarray()

In [ ]:
print (count_vectorizer_data_features.shape)
print(count_vectorizer_data_features)

**Vocabulary**

In [ ]:
# Take a look at the words in the vocabulary
count_vectorizer_vocab = count_vectorizer.get_feature_names()
print (count_vectorizer_vocab)

**Stop words**

In [ ]:
count_vectorizer_stop_words = count_vectorizer.get_stop_words()
print (count_vectorizer_stop_words)
# There are no stop words since we are doing sentiment analysis

**Sum up the counts of each vocabulary word**

In [ ]:
# Sum up the counts of each vocabulary word
dist = np.sum(count_vectorizer_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(count_vectorizer_vocab, dist):
    print (count, tag)

**TfidfVectorizer**

In [ ]:
# Initialize the "TfidfVectorizer" object
# Convert a collection of raw documents to a matrix of TF-IDF features.
tfidf_vectorizer = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000,
                             min_df=5, 
                             max_df=0.7,
                             ngram_range=(1,2)) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
tfidf_vectorizer_data_features = tfidf_vectorizer.fit_transform(clean_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
tfidf_vectorizer_data_features = tfidf_vectorizer_data_features.toarray()

In [ ]:
print (tfidf_vectorizer_data_features.shape)
print(tfidf_vectorizer_data_features)

**Vocabulary**

In [ ]:
# Take a look at the words in the vocabulary
tfidf_vectorizer_vocab = tfidf_vectorizer.get_feature_names()
print (tfidf_vectorizer_vocab)

**Stop Words**

In [ ]:
tfidf_vectorizer_stop_words = tfidf_vectorizer.get_stop_words()
print (tfidf_vectorizer_stop_words)

**Sum up the counts of each vocabulary word**

In [ ]:
# Sum up the counts of each vocabulary word
tf_df_dist = np.sum(tfidf_vectorizer_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(tfidf_vectorizer_vocab, tf_df_dist):
    print (count, tag)

# <font color='blue'>Step 5</font>

**Fit and evaluate model using both type of vectorization. **

**RandomForest Classifier on CountVectorizer**

In [ ]:
tweet_data_relevant.head()

In [ ]:
y = tweet_data_relevant['airline_sentiment']
print(y.value_counts())
plt.figure(figsize=(20,5)) 
g = sns.countplot(y)
# No, Here we can see that the data is not balanced, There are lot of negative sentiments

**Dividing Data to Train and Test**

In [ ]:
x = count_vectorizer_data_features     # Predictor feature columns
y = tweet_data_relevant['airline_sentiment']   # Predicted class

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)  # 1 is just any random seed number

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

**Applying SMOTE since the data is not balanced**

In [ ]:
smt = SMOTE(random_state=0)
X_train_SMOTE, y_train_SMOTE = smt.fit_sample(x_train, y_train)

In [ ]:
print(X_train_SMOTE.shape)
print(y_train_SMOTE.shape)

In [ ]:
y_train_SMOTE

**Checking if data is balanced after applying SMOTE**

In [ ]:
after_smote_airline_sentiment=pd.DataFrame(y_train_SMOTE, columns=['airline_sentiment']) 
y = after_smote_airline_sentiment['airline_sentiment']
print(y.value_counts())
plt.figure(figsize=(20,5)) 
g = sns.countplot(y)
# Here we can see that after applying smote, the data is balanced

**Dividing Test data to Test and Validation Data**

In [ ]:
# Dividing the test data into test and validation set in 50-50 ratio
x_validation, x_test_main, y_validation, y_test_main = train_test_split(x_test, y_test, test_size=0.50, random_state=1)

print(x_validation.shape)
print(x_test_main.shape)


print(y_validation.shape)
print(y_test_main.shape)

# There are 2185 samples for validation and 2186 samples for testing

**Initialize RandomForestClassifier**

In [ ]:
# Initialize a Random Forest classifier with 100 trees
randomforestclassifier = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False) 
# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable

print ("Training the random forest...")
randomforestclassifier = randomforestclassifier.fit( X_train_SMOTE, y_train_SMOTE)

In [ ]:
randomforestclassifier.score(X_train_SMOTE, y_train_SMOTE)

**Evaluate score by cross-validation**

In [ ]:
print (np.mean(cross_val_score(randomforestclassifier,X_train_SMOTE, y_train_SMOTE,cv=10)))

In [ ]:
# Make class predictions for the Validation set
y_validation_predict= randomforestclassifier.predict(x_validation)

**Training and Validation Accuracy**

In [ ]:
print("Trainig accuracy",randomforestclassifier.score(X_train_SMOTE,y_train_SMOTE))  
print()
print("Validation accuracy",randomforestclassifier.score(x_validation, y_validation))
print()

**Classification Report**  (Validation Set)

In [ ]:
print(metrics.classification_report(y_validation,y_validation_predict))

**Confusion Matrix** (Validation Set)

In [ ]:
cm=confusion_matrix(y_validation_predict , y_validation)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True,cmap=plt.cm.Reds)
plt.xticks(range(2), ['Negative', 'Neutral', 'Positive'], fontsize=16,color='black')
plt.yticks(range(2), ['Negative', 'Neutral', 'Positive'], fontsize=16)
plt.show()

**Testing Data Accuracy**

In [ ]:
print("Testing accuracy",randomforestclassifier.score(x_test_main, y_test_main))
print()

**Classification Report**  (Test Set)

In [ ]:
# Make class predictions for the Validation set
y_test_predict= randomforestclassifier.predict(x_test_main)

print(metrics.classification_report(y_test_main,y_test_predict))

**Confusion Matrix** (Test Set)

In [ ]:
cm=confusion_matrix(y_test_predict , y_test_main)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True,cmap=plt.cm.Reds)
plt.xticks(range(2), ['Negative', 'Neutral', 'Positive'], fontsize=16,color='black')
plt.yticks(range(2), ['Negative', 'Neutral', 'Positive'], fontsize=16)
plt.show()

**Random Forest Classifier on TfidfVectorizer**

**Dividing Data into train and Test**

In [ ]:
x_tf_idf = tfidf_vectorizer_data_features     # Predictor feature columns
y_tf_idf = tweet_data_relevant['airline_sentiment']   # Predicted class

x_train_tf_idf, x_test_tf_idf, y_train_tf_idf, y_test_tf_idf = train_test_split(x_tf_idf, y_tf_idf, test_size=0.3, random_state=1)  # 1 is just any random seed number

In [ ]:
print(x_train_tf_idf.shape)
print(y_train_tf_idf.shape)
print(x_test_tf_idf.shape)
print(y_test_tf_idf.shape)

In [ ]:
y = tweet_data_relevant['airline_sentiment']
print(y.value_counts())
plt.figure(figsize=(20,5)) 
g = sns.countplot(y)
# No, Here we can see that the data is not balanced, There are lot of negative sentiments

**Applying SMOTE since the data is not balanced**

In [ ]:
tf_idf_smt = SMOTE(random_state=0)
X_train_tf_idf_SMOTE, y_train_tf_idf_SMOTE = smt.fit_sample(x_train_tf_idf, y_train_tf_idf)

In [ ]:
print(X_train_tf_idf_SMOTE.shape)
print(y_train_tf_idf_SMOTE.shape)

In [ ]:
after_smote_airline_sentiment_tf_idf=pd.DataFrame(y_train_tf_idf_SMOTE, columns=['airline_sentiment']) 
y = after_smote_airline_sentiment_tf_idf['airline_sentiment']
print(y.value_counts())
plt.figure(figsize=(20,5))
g = sns.countplot(y)
# Here we can see that after smote , the data is balanced

In [ ]:
# Dividing the test data into test and validation set in 50-50 ratio
x_validation_tf_idf, x_test_main_tf_idf, y_validation_tf_idf, y_test_main_tf_idf = train_test_split(x_test_tf_idf, y_test_tf_idf, test_size=0.50, random_state=1)

print(x_validation_tf_idf.shape)
print(x_test_main_tf_idf.shape)


print(y_validation_tf_idf.shape)
print(y_test_main_tf_idf.shape)

# There are 2185 samples for validation and 2186 samples for testing

**Initialize RandomForestClassifier**

In [ ]:
# Initialize a Random Forest classifier with 100 trees
randomforestclassifier_tf_idf = RandomForestClassifier(verbose=1,n_jobs=-1,n_estimators = 100) 
# Fit the forest to the training set

print ("Training the random forest...")
randomforestclassifier_tf_idf = randomforestclassifier_tf_idf.fit( X_train_tf_idf_SMOTE, y_train_tf_idf_SMOTE)

In [ ]:
randomforestclassifier_tf_idf.score(X_train_tf_idf_SMOTE, y_train_tf_idf_SMOTE)

**Evaluate score by cross-validation**

In [ ]:
print (np.mean(cross_val_score(randomforestclassifier_tf_idf,X_train_tf_idf_SMOTE, y_train_tf_idf_SMOTE,cv=10)))

In [ ]:
# Make class predictions for the Validation set
y_validation_predict_tf_idf= randomforestclassifier_tf_idf.predict(x_validation_tf_idf)

**Training and Validation Accuracy**

In [ ]:
print("Trainig accuracy",randomforestclassifier_tf_idf.score(X_train_tf_idf_SMOTE,y_train_tf_idf_SMOTE))  
print()
print("Validation accuracy",randomforestclassifier_tf_idf.score(x_validation_tf_idf, y_validation_tf_idf))
print()

**Classification Report**  (Validation Set)

In [ ]:
print(metrics.classification_report(y_validation_tf_idf,y_validation_predict_tf_idf))

**Confusion Matrix** (Validation Set)

In [ ]:
cm=confusion_matrix(y_validation_predict_tf_idf , y_validation_tf_idf)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True,cmap=plt.cm.Reds)
plt.xticks(range(2), ['Negative', 'Neutral', 'Positive'], fontsize=16,color='black')
plt.yticks(range(2), ['Negative', 'Neutral', 'Positive'], fontsize=16)
plt.show()

**Test Accuracy**

In [ ]:
print("Test accuracy",randomforestclassifier_tf_idf.score(x_test_main_tf_idf, y_test_main_tf_idf))
print()

**Classification Report**  (Test Set)

In [ ]:
# Make class predictions for the test set
y_test_predict_tf_idf= randomforestclassifier_tf_idf.predict(x_test_main_tf_idf)
print(metrics.classification_report(y_test_main_tf_idf,y_test_predict_tf_idf))

**Confusion Matrix** (Test Set)

In [ ]:
cm=confusion_matrix(y_test_predict_tf_idf , y_test_main_tf_idf)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True,cmap=plt.cm.Reds)
plt.xticks(range(2), ['Negative', 'Neutral', 'Positive'], fontsize=16,color='black')
plt.yticks(range(2), ['Negative', 'Neutral', 'Positive'], fontsize=16)
plt.show()

# <font color='blue'>Step 6</font>

**Summarize your understanding of the application of Various Pre-processing and Vectorization and
performance of your model on this dataset.**


**Summary**

---



**Pre-processing steps in NLP**

In [ ]:
# Pre-processing steps in NLP
#      1. HTML tag cleanup
#                          -  It returns all the text in a document or beneath a tag, as a single Unicode string:
#      2. Contraction
#                          - Contractions are shortened version of words or syllables. 
#                          - In case of English contractions are often created by removing one of the vowels from the word. 
#                          - Examples would be, do not to don’t and I would to I’d. Converting each contraction to its expanded, original form helps with text standardization.
#      3. Tokenization
#                          - Tokenization is a step which splits longer strings of text into smaller pieces, or tokens. 
#                          - Larger chunks of text can be tokenized into sentences, sentences can be tokenized into words, etc. 
#                          - Further processing is generally performed after a piece of text has been appropriately tokenized. 
#                          - Tokenization is also referred to as text segmentation or lexical analysis.
#                          - Sometimes segmentation is used to refer to the breakdown of a large chunk of text into pieces larger than words (e.g. paragraphs or sentences), 
#                                        while tokenization is reserved for the breakdown process which results exclusively in words.
#      4. Removing numbers
#                          - Remove numbers from list of tokenized words
#      5. Remove special characters
#                          - Remove special characters from list of tokenized words
#      6. Remove punctuation
#                          - Remove punctuation from list of tokenized words
#      7. Convert text to lower case
#                          - converting all text to the same case 
#      8. Remove empty strings
#                          - Remove empty string from list of tokenized words
#      9. Stemming
#                          - Converting the words into their base word or stem word ( Ex - tastefully, tasty, these words are converted to stem word called 'tasti'). 
#                            This reduces the vector dimension because we dont consider all similar words
#      10.Lemmatization
#                          - Lemmatization, unlike Stemming, reduces the inflected words properly ensuring that the root word belongs to the language. 
#                            In Lemmatization root word is called Lemma. A lemma (plural lemmas or lemmata) is the canonical form, dictionary form, or citation form of a set of words.

**Techniques for Encoding**

In [ ]:
# Steps to do after text pre-processing:
#Techniques for Encoding - These are the popular techniques that are used for encoding:

#       o Bag of words         (CountVectorization)
#                              In BoW we construct a dictionary that contains set of all unique words from our text review dataset. 
#                              The frequency of the word is counted here. If there are d unique words in our dictionary then for every sentence or review the vector will be of length d 
#                              and count of word from review is stored at its particular location in vector. The vector will be highly sparse in such case.
#       o Tf-idf               (TfIdfVectorization)  (Term Frequency - Inverse Document Frequency)
#                               Term Frequency - Inverse Document Frequency it makes sure that less importance is given to most frequent words and also considers less frequent words.
#                               Term Frequency is number of times a particular word(W) occurs in a review divided by totall number of words (Wr) in review. The term frequency value ranges from 0 to 1.
#                               Inverse Document Frequency is calculated as log(Total Number of Docs(N) / Number of Docs which contains particular word(n)). Here Docs referred as Reviews.
#                               TF-IDF is TF * IDF that is (W/Wr)*LOG(N/n)

**Performance of Classification Model**

**RandomForestClassifier on CountVectorizer**

In [ ]:
print("Trainig accuracy",randomforestclassifier.score(X_train_SMOTE,y_train_SMOTE))  
print()
print("Testing accuracy",randomforestclassifier.score(x_test_main, y_test_main))
print()

y_test_predict= randomforestclassifier.predict(x_test_main)
print(metrics.classification_report(y_test_main,y_test_predict))

cm=confusion_matrix(y_test_predict , y_test_main)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True,cmap=plt.cm.Reds)
plt.xticks(range(2), ['Negative', 'Neutral', 'Positive'], fontsize=16,color='black')
plt.yticks(range(2), ['Negative', 'Neutral', 'Positive'], fontsize=16)
plt.show()

**RandomForestClassifier on TfidfVectorizer**

In [ ]:
print("Trainig accuracy",randomforestclassifier_tf_idf.score(X_train_tf_idf_SMOTE,y_train_tf_idf_SMOTE))  
print()
print("Test accuracy",randomforestclassifier_tf_idf.score(x_test_main_tf_idf, y_test_main_tf_idf))
print()

# Make class predictions for the test set
y_test_predict_tf_idf= randomforestclassifier_tf_idf.predict(x_test_main_tf_idf)
print(metrics.classification_report(y_test_main_tf_idf,y_test_predict_tf_idf))

cm=confusion_matrix(y_test_predict_tf_idf , y_test_main_tf_idf)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True,cmap=plt.cm.Reds)
plt.xticks(range(2), ['Negative', 'Neutral', 'Positive'], fontsize=16,color='black')
plt.yticks(range(2), ['Negative', 'Neutral', 'Positive'], fontsize=16)
plt.show()

In [ ]:
Accuracy=[]
Model=[]
Accuracy.append(randomforestclassifier.score(x_test_main, y_test_main))
Accuracy.append(randomforestclassifier_tf_idf.score(x_test_main_tf_idf, y_test_main_tf_idf))
Model.append("RandomForestClassifier on CountVectorizer")
Model.append("RandomForestClassifier on TfidfVectorizer")


index=[0,1]
plt.bar(index,Accuracy,color='rgbyk')
plt.xticks(index,Model,rotation=45)
plt.ylabel('Accuracy')
plt.xlabel('Model')
plt.title('Classifier Accuracies')

xlocs, xlabs = plt.xticks()

for i, v in enumerate(Accuracy):
    plt.text(xlocs[i] - 0.25, v + 0.01, str(v))

# The RandomForestClassfier on TfidfVectorizer is having better accuracy

**Summary**

---





*   RandomForestClassifier on TfidfVectorizer has an accuracy of 77% whereas 
RandomForestClassifier on CountVectorizer has an accuracy of 75%. This shows that the model using TfidfVectorizer is better.
*   Model is able to predict airline sentiment correctly 77% of the time
*   United Airlines, US Airways, American Airlines has the most number of negative review
*   Virgin America has the least number of negative reviews
*   Customer service and Late flight seems to be the main reason why customers are giving bad feedback


